In [1]:
import json
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import os
import re
import cv2

## Edit the JS file each frame ##

In [2]:
URL_BASE = 'http://0.0.0.0:31338/'
MOVIE_FOLDER_CYCLES = 'CycleEdgeData_level_-3/'
MOVIE_FOLDER_RIPS_EDGES = 'RipsGraphEdges_level_-3/'
MOVIE_FOLDER_RIPS_NODES = 'RipsGraphNodes_level_-3/'
MOVIE_FOLDER_CONN = 'H0Gens_level_-3/'

In [3]:
with open('cyclePlotBase.js','r') as f:
    cycle_orig_lines = f.readlines()
    
with open('ripsPlotBase.js','r') as f:
    rips_orig_lines = f.readlines()
    
with open('H0PlotBase.js','r') as f:
    H0_orig_lines = f.readlines()

cycle_lines = cycle_orig_lines.copy()
rips_lines = rips_orig_lines.copy()
H0_lines = H0_orig_lines.copy()
arr_line_ind = [i for i,line in enumerate(cycle_orig_lines) if re.findall(r'REPLACE_FILE_ARR',line)][0]

In [4]:
def sort_key(in_str):
    digits = tuple([int(x) for x in re.findall(r'\d+',in_str)])
    return digits

In [5]:
files_cycles = sorted(os.listdir(MOVIE_FOLDER_CYCLES),key=sort_key)
files_rips_edges = sorted(os.listdir(MOVIE_FOLDER_RIPS_EDGES),key=sort_key)
files_rips_nodes = sorted(os.listdir(MOVIE_FOLDER_RIPS_NODES),key=sort_key)
files_conn = sorted(os.listdir(MOVIE_FOLDER_CONN),key=sort_key)

## Get generator array for each frame ##

In [6]:
frames_cycles = sorted(set([re.findall(r'\d+',file)[0] for file in files_cycles]),key = lambda x: int(x))
frames_rips_edges = sorted(set([re.findall(r'\d+',file)[0] for file in files_rips_edges]),key = lambda x: int(x))
frames_rips_nodes = sorted(set([re.findall(r'\d+',file)[0] for file in files_rips_nodes]),key = lambda x: int(x))
frames_conn = sorted(set([re.findall(r'\d+',file)[1] for file in files_conn]),key = lambda x: int(x))
gen_cycles = [(frame,sorted([URL_BASE+MOVIE_FOLDER_CYCLES+file for file in files_cycles if re.findall(r'\d+',file)[0] == frame],key=sort_key)) for frame in frames_cycles]
gen_rips_edges = [(frame,sorted([URL_BASE+MOVIE_FOLDER_RIPS_EDGES+file for file in files_rips_edges if re.findall(r'\d+',file)[0] == frame],key=sort_key)) for frame in frames_rips_edges]
gen_rips_nodes = [(frame,sorted([URL_BASE+MOVIE_FOLDER_RIPS_NODES+file for file in files_rips_nodes if re.findall(r'\d+',file)[0] == frame],key=sort_key)) for frame in frames_rips_nodes]
gen_conn = [(frame,sorted([URL_BASE+MOVIE_FOLDER_CONN+file for file in files_conn if re.findall(r'\d+',file)[1] == frame],key=sort_key)) for frame in frames_conn]

In [7]:
num_frames = len(os.listdir('TotPers_level_-3'))

In [8]:
num_frames

388

In [40]:
with open('map.html','r') as f:
    map_orig_lines = f.readlines()
    
map_lines = map_orig_lines.copy()
TR_ind = [i for i,line in enumerate(map_orig_lines) if re.findall(r'TOP_RIGHT_IMAGE',line)][0]
MR_ind = [i for i,line in enumerate(map_orig_lines) if re.findall(r'MID_RIGHT_IMAGE',line)][0]
BR_ind = [i for i,line in enumerate(map_orig_lines) if re.findall(r'BOT_RIGHT_IMAGE',line)][0]

In [41]:
browser = webdriver.Chrome()
browser.set_window_size(600,1000)

for frame in range(0,num_frames):
    TR_IM = 'TotPers_level_-3/TotPers_frame_'+str(frame)+'.png'
    MR_IM = 'H0Plots_level_-3/H0Plots_frame_'+str(frame)+'.png'
    BR_IM = 'H1Pers_level_-3/H1Pers_frame_'+str(frame)+'.png'
    
    map_lines[TR_ind] = map_orig_lines[TR_ind].replace(r'TOP_RIGHT_IMAGE',TR_IM)
    map_lines[MR_ind] = map_orig_lines[MR_ind].replace(r'MID_RIGHT_IMAGE',MR_IM)
    map_lines[BR_ind] = map_orig_lines[BR_ind].replace(r'BOT_RIGHT_IMAGE',BR_IM)

    with open('mapTemp.html','w+') as f:
        for line in map_lines:
            f.write(line)
            
    cycle_arr = [gen for fr,gen in gen_cycles if int(fr) == frame]
    rips_arr = [gen for fr,gen in gen_rips_edges if int(fr) == frame] + [gen for fr,gen in gen_rips_nodes if int(fr) == frame]
    H0_arr = [gen for fr,gen in gen_conn if int(fr) == frame]
    
    if cycle_arr:
        cycle_lines[arr_line_ind] = cycle_orig_lines[arr_line_ind].replace('REPLACE_FILE_ARR',repr(cycle_arr[0]))
    else:
        cycle_lines[arr_line_ind] = cycle_orig_lines[arr_line_ind].replace('REPLACE_FILE_ARR','["' + URL_BASE + 'placeholder.json'+ '"]')
    if rips_arr:
        rips_lines[arr_line_ind] = rips_orig_lines[arr_line_ind].replace('REPLACE_FILE_ARR',repr(rips_arr))
    else:
        rips_lines[arr_line_ind] = rips_orig_lines[arr_line_ind].replace('REPLACE_FILE_ARR','["'+URL_BASE + 'placeholder.json'+'"]')
    if H0_arr:
        H0_lines[arr_line_ind] = H0_orig_lines[arr_line_ind].replace('REPLACE_FILE_ARR',repr(H0_arr[0]))
    else:
        H0_lines[arr_line_ind] = H0_orig_lines[arr_line_ind].replace('REPLACE_FILE_ARR','["'+URL_BASE + 'placeholder.json'+'"]')
        
    with open('cyclePlotTemp.js','w+') as f:
        for line in cycle_lines:
            f.write(line)
    with open('ripsPlotTemp.js','w+') as f:
        for line in rips_lines:
            f.write(line)
    with open('H0PlotTemp.js','w+') as f:
        for line in H0_lines:
            f.write(line)
            
    browser.get("file://"+os.getcwd()+"/"+"mapTemp.html")
    delay = 10
    try:
        waitElemCycle = WebDriverWait(browser,delay).until(EC.presence_of_element_located((By.ID, 'waitDivCycle')))
        waitElemRips = WebDriverWait(browser,delay).until(EC.presence_of_element_located((By.ID, 'waitDivRips')))
        waitElemH0 = WebDriverWait(browser,delay).until(EC.presence_of_element_located((By.ID, 'waitDivH0')))
    except TimeoutException:
        print("Loading took too long")
    browser.save_screenshot("MovieImages/im_min"+str(frame)+".png")
            
            

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=80.0.3987.122)
  (Driver info: chromedriver=2.41.578706 (5f725d1b4f0a4acbf5259df887244095596231db),platform=Mac OS X 10.13.6 x86_64)


In [116]:
image_folder = 'MovieImages'
video_name = 'TestMovie_level_-3.mp4'

images = sorted([img for img in os.listdir(image_folder) if img.endswith(".png")],key=sort_key)
good_images = [img for img in images if int(re.findall(r'\d+',img)[0]) < num_frames]
frame = cv2.imread(os.path.join(image_folder, good_images[0]))
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_name, fourcc, 20, (width,height))

for image in good_images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()